In [1]:
# budget != null && revuenue != null
# release_date >= 1990


In [2]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
import plotly
import plotly.offline as py
sb.set() 


Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'plotly'

In [ ]:
df = pd.read_csv('tmdb_5000_movies.csv')
df.head().transpose()

In [ ]:
df.shape

In [ ]:
df

---

## Data Cleaning



In [ ]:
data = df
data

In [ ]:
df[df['budget'] == 0].shape

In [ ]:
df[df['revenue'] == 0].shape

#### Remove rows that "genres" or "keywords" is empty

In [ ]:
data = data[data['genres'] !='[]']
data = data[data['keywords'] !='[]']
data

### Production Countries

Remove data where "production_countries" is empty

In [ ]:
data = data[data['production_countries'] !='[]']

Form data with country name and number of movies

In [ ]:
s = df.apply(lambda x: pd.Series(x['production_countries']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'countries'

In [ ]:
con_df = df.drop('production_countries', axis=1).join(s)
con_df = pd.DataFrame(con_df['countries'].value_counts())
con_df['country'] = con_df.index
con_df.columns = ['num_movies', 'country']
con_df = con_df.reset_index().drop('index', axis=1)
con_df.head(20)

### Remove production country of USA 

More than 90% of movies are production in USA. In order to make other counties more clear in map, we need to remove USA from the list

In [ ]:
con_df[con_df['country'] == 'United States of America']['num_movies'] / len(data)

In [ ]:
con_df = con_df[con_df['country'] != 'United States of America']

In [ ]:
data = [ dict(
        type = 'choropleth',
        locations = con_df['country'],
        locationmode = 'country names',
        z = con_df['num_movies'],
        text = con_df['country'],
        colorscale = [[0,'rgb(255, 255, 255)'],[1,'rgb(255, 0, 0)']],
        autocolorscale = False,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Production Countries'),
      ) ]

layout = dict(
    title = 'Production Countries for the Movies (Exclude from US)',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )